In [14]:
import osmnx as ox
import folium
import pandas as pd
import geopandas as gpd
from shapely import wkt 
from shapely.geometry import Polygon, MultiPolygon
import json

# Función para buscar parques, jardines y calles en una ubicación específica.
def buscar_parques_jardines_y_calles():
    lugar = "Burgos, Spain"

    # Extraer características geográficas de parques y jardines de OSM en la ubicación dada.
    parques_y_jardines = ox.features_from_place(lugar, {
        'leisure': ['park', 'nature_reserve', 'dog_park'],
        'landuse': ['forest', "village_green", "grass"],
        'natural': ['scrub', 'heath']
    })

    # Extraer características de jardines específicamente.
    jardines = ox.features_from_place(lugar, {'leisure': 'garden'})

    # Filtrar jardines con nombres.
    jardines_con_nombre = jardines[jardines['name'].notna()]

    # Combinar los datasets de parques y jardines en un GeoDataFrame.
    parques_y_jardines = gpd.GeoDataFrame(pd.concat([parques_y_jardines, jardines_con_nombre], ignore_index=True))
    
    # Extraer características de calles de OSM en la ubicación dada.
    calles = ox.features_from_place(lugar, {
        'highway': ['footway', 'path', 'pedestrian', 'living_street', 'track', 'primary', 'secondary', 'tertiary', 'residential', ]
    })

    return parques_y_jardines, calles

# Función para agregar columnas de tipo y ID a un GeoDataFrame.
def agregar_columna_tipo_y_id(dataframe, tipo_columna, es_calle=False):
    # Asignar tipo de calle o tipo de jardín basado en los atributos del GeoDataFrame.
    if es_calle:
        dataframe['tipo_calle'] = dataframe['highway']
    else:
        dataframe[tipo_columna] = dataframe.apply(lambda x: x['leisure'] if pd.notna(x['leisure']) else 
                                                (x['landuse'] if pd.notna(x['landuse']) else x['natural']), axis=1)
    # Asignar un ID único a cada fila.
    dataframe['id'] = range(1, len(dataframe) + 1)

    # Calcular el área de cada geometría y luego reconvertir las coordenadas a EPSG:4326.
    dataframe['area'] = dataframe.to_crs(epsg=3857).area  
    dataframe = dataframe.to_crs(epsg=4326)  

    return dataframe

# Función para convertir listas y diccionarios a strings en un DataFrame.
def convertir_a_string_si_es_necesario(valor):
    if isinstance(valor, (list, dict)):
        return json.dumps(valor)
    return valor

# Función para preparar un DataFrame para ser exportado a un archivo GeoJSON.
def preparar_para_geojson(dataframe):
    for columna in dataframe.columns:
        if columna != 'geometry':  
            dataframe[columna] = dataframe[columna].apply(convertir_a_string_si_es_necesario)
    return dataframe

# Función para guardar los DataFrames como archivos CSV y GeoJSON.
def guardar_dataframes_en_csv_y_geojson(parques_y_jardines, calles):
    parques_y_jardines_preparados = preparar_para_geojson(parques_y_jardines.copy())
    calles_preparados = preparar_para_geojson(calles.copy())

    # Guardar en formato CSV.
    parques_y_jardines_preparados.to_csv('parques_y_jardines.csv', index=False)
    calles_preparados.to_csv('calles.csv', index=False)

    # Guardar en formato GeoJSON.
    parques_y_jardines_preparados.to_file('parques_y_jardines.geojson', driver='GeoJSON')
    calles_preparados.to_file('calles.geojson', driver='GeoJSON')

# Función para ajustar polígonos superpuestos en un GeoDataFrame.
def ajustar_poligonos_superpuestos(dataframe):
    poligonos_ajustados = []

    for i, area1 in dataframe.iterrows():
        poligono_ajustado = area1.geometry
        for _, area2 in dataframe.iterrows():
            if area1.id != area2.id and area1.geometry.intersects(area2.geometry):
                poligono_ajustado = poligono_ajustado.difference(area2.geometry)

        if isinstance(poligono_ajustado, (Polygon, MultiPolygon)):
            poligonos_ajustados.append(poligono_ajustado)
        else:
            poligonos_ajustados.append(area1.geometry)  

    return poligonos_ajustados

# Función para agregar polígonos a un mapa de Folium.
def agregar_poligonos_al_mapa(mapa, dataframe, color, opacidad):
    for area in dataframe.itertuples():
        if not isinstance(area.geometry, (Polygon, MultiPolygon)) or area.geometry.is_empty:
            continue

        # Crear y agregar un popup con información sobre el área.
        nombre = getattr(area, 'name', 'Sin nombre')
        popup_text = f"ID: {area.id}<br>Nombre: {nombre}<br>Área: {area.area:.2f} m²"
        popup = folium.Popup(popup_text, parse_html=True)
        geojson = folium.GeoJson(
            area.geometry.__geo_interface__,
            style_function=lambda feature: {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': opacidad
            }
        )
        geojson.add_child(popup)
        geojson.add_to(mapa)

# Función para agregar líneas (calles) a un mapa de Folium.
def agregar_lineas_al_mapa(mapa, dataframe, color):
    for _, calle in dataframe.iterrows():
        if calle.geometry is None or calle.geometry.is_empty:
            continue

        # Agregar las geometrías de línea al mapa.
        if calle.geometry.geom_type == 'LineString' or calle.geometry.geom_type == 'MultiLineString':
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,
                    'weight': 2
                }
            ).add_to(mapa)

# Función para crear un mapa de Folium con elementos geográficos (polígonos o líneas).
def crear_mapa_con_elementos(dataframe, tipo_elemento, color, opacidad=None):
    # Inicializar un mapa de Folium.
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

    # Agregar polígonos o líneas al mapa según el tipo de elemento.
    if tipo_elemento == 'poligono':
        # Código específico para manejar parques caninos.
        if 'tipo_jardin' in dataframe.columns:
            non_dog_parks = dataframe[dataframe['tipo_jardin'] != 'dog_park']
            agregar_poligonos_al_mapa(mapa, non_dog_parks, color, opacidad)

            dog_parks = dataframe[dataframe['tipo_jardin'] == 'dog_park']
            for _, dog_park in dog_parks.iterrows():
                if dog_park.geometry.geom_type in ['Polygon', 'MultiPolygon']:
                    popup_text = f"ID: {dog_park['id']}<br>Parque Canino: {dog_park.get('name', 'Sin nombre')}"
                    popup = folium.Popup(popup_text, parse_html=True)
                    folium.GeoJson(
                        dog_park.geometry.__geo_interface__,
                        style_function=lambda feature: {
                            'fillColor': 'red',  
                            'color': 'red',
                            'weight': 2,
                            'fillOpacity': 0.6}
                    ).add_child(popup).add_to(mapa)
        else:
            agregar_poligonos_al_mapa(mapa, dataframe, color, opacidad)

    elif tipo_elemento == 'linea':
        agregar_lineas_al_mapa(mapa, dataframe, color)
    
    return mapa

# Ejecutar las funciones para obtener y procesar los datos.
parques_y_jardines, calles = buscar_parques_jardines_y_calles()

# Convertir los datos en GeoDataFrames.
parques_y_jardines_gdf = gpd.GeoDataFrame(parques_y_jardines, crs="EPSG:4326")
calles_gdf = gpd.GeoDataFrame(calles, crs="EPSG:4326")

# Agregar columnas y calcular áreas.
parques_y_jardines_gdf = agregar_columna_tipo_y_id(parques_y_jardines_gdf, 'tipo_jardin')
calles_gdf = agregar_columna_tipo_y_id(calles_gdf, 'tipo_calle', es_calle=True)

# Guardar los datos en formatos.
guardar_dataframes_en_csv_y_geojson(parques_y_jardines_gdf, calles_gdf)

# Crear mapa de los parques.
mapa_parques = crear_mapa_con_elementos(parques_y_jardines_gdf, 'poligono', 'green', 0.4)
mapa_parques.save('mapa_parques.html')

# Crear mapa de las calles.
mapa_calles = crear_mapa_con_elementos(calles_gdf, 'linea', 'blue')
mapa_calles.save('mapa_calles.html')


Columnas después de agregar 'tipo_jardin': Index(['barrier', 'geometry', 'access', 'natural', 'source', 'name',
       'addr:city', 'addr:postcode', 'addr:street', 'operator',
       'operator:type', 'website', 'wheelchair', 'wikidata', 'wikipedia',
       'leisure', 'nodes', 'landuse', 'leaf_type', 'designation', 'name:1905',
       'name:1944', 'description', 'produce', 'place', 'surface', 'loc_name',
       'tourism', 'old_name', 'alt_name', 'type', 'grass', 'start_date',
       'phone', 'material', 'wall', 'ways', 'fee', 'source:date',
       'garden:type', 'building', 'traffic_calming', 'wikimedia_commons',
       'area', 'tipo_jardin', 'id'],
      dtype='object')
Columnas después de agregar 'tipo_calle': Index(['button_operated', 'crossing', 'highway', 'geometry', 'bus', 'name',
       'tactile_paving', 'traffic_signals:sound', 'traffic_signals:vibration',
       'crossing:island',
       ...
       'parking:condition:left:time_interval',
       'parking:condition:right:maxstay'

In [12]:
def agregar_lineas_sin_nombre_al_mapa(mapa, dataframe, color):
    # Itera sobre cada fila (calle) en el dataframe.
    for _, calle in dataframe.iterrows():
        # Si la geometría de la calle es nula, está vacía o si la calle tiene un nombre, salta a la siguiente iteración.
        if calle.geometry is None or calle.geometry.is_empty or pd.notna(calle.get('name')):
            continue

        # Verifica si la geometría de la calle es de tipo 'LineString' o 'MultiLineString'.
        if calle.geometry.geom_type in ['LineString', 'MultiLineString']:
            # Crea un objeto GeoJson con la geometría de la calle y lo agrega al mapa.
            # El estilo del GeoJson se define con el color y grosor de la línea.
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,
                    'weight': 2
                }
            ).add_to(mapa)

def crear_mapa_calles_sin_nombre(dataframe, color):
    # Crea un objeto mapa de folium centrado en una ubicación específica y con un nivel de zoom inicial.
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

    # Llama a la función 'agregar_lineas_sin_nombre_al_mapa' para añadir las calles sin nombre al mapa.
    agregar_lineas_sin_nombre_al_mapa(mapa, dataframe, color)

    # Devuelve el objeto mapa.
    return mapa

# Filtra el GeoDataFrame 'calles_gdf' para obtener solo las calles que no tienen un nombre asignado.
calles_sin_nombre_gdf = calles_gdf[calles_gdf['name'].isna()]

# Crea un mapa con las calles sin nombre utilizando la función 'crear_mapa_calles_sin_nombre'.
# Se utiliza el color azul ('blue') para las líneas que representan estas calles.
mapa_calles_sin_nombre = crear_mapa_calles_sin_nombre(calles_sin_nombre_gdf, 'blue')

# Guarda el mapa en un archivo HTML para su visualización.
mapa_calles_sin_nombre.save('mapa_calles_sin_nombre.html')


In [13]:
def agregar_lineas_con_nombre_al_mapa(mapa, dataframe, color):
    # Itera sobre cada calle en el dataframe.
    for _, calle in dataframe.iterrows():
        # Omite la calle si su geometría es nula, está vacía o no tiene un nombre.
        if calle.geometry is None or calle.geometry.is_empty or pd.isna(calle.get('name')):
            continue

        # Verifica si la geometría de la calle es de tipo 'LineString' o 'MultiLineString'.
        if calle.geometry.geom_type in ['LineString', 'MultiLineString']:
            # Crea un popup con el nombre de la calle.
            popup = folium.Popup(calle['name'], parse_html=True)

            # Crea un objeto GeoJson para la calle y lo agrega al mapa. Personaliza el estilo de la línea.
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,  # Color de la línea.
                    'weight': 2      # Grosor de la línea.
                }
            ).add_child(popup).add_to(mapa)  # Añade el popup a la línea y la línea al mapa.

def crear_mapa_calles_con_nombre(dataframe, color):
    # Crea un nuevo mapa de Folium centrado en una ubicación y con un nivel de zoom inicial.
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

    # Llama a la función anterior para añadir las calles con nombre al mapa.
    agregar_lineas_con_nombre_al_mapa(mapa, dataframe, color)

    # Devuelve el objeto mapa.
    return mapa

# Filtra el GeoDataFrame 'calles_gdf' para obtener solo las calles que tienen un nombre asignado.
calles_con_nombre_gdf = calles_gdf[calles_gdf['name'].notna()]

# Utiliza la función 'crear_mapa_calles_con_nombre' para crear un mapa con estas calles.
# Se usa el color verde ('green') para representar las calles.
mapa_calles_con_nombre = crear_mapa_calles_con_nombre(calles_con_nombre_gdf, 'green')

# Guarda el mapa resultante en un archivo HTML para su visualización.
mapa_calles_con_nombre.save('mapa_calles_con_nombre.html')